In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 #
block_size = 256 #
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 10
n_layer = 10
dropout = 0.2
torch.manual_seed(1337)
with open('/content/drive/MyDrive/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        perplexities = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss, perplexity = model(X, Y)
            losses[k] = loss.item()
            perplexities[k] = perplexity.item() if perplexity is not None else float('nan')
        out[f'{split}_loss'] = losses.mean()
        out[f'{split}_perplexity'] = perplexities.mean()
    model.train()
    return out

def save_checkpoint(model, optimizer, iter, filepath):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'iteration': iter
    }
    torch.save(checkpoint, filepath)
    print(f"Model checkpoint saved at iteration {iter}.")


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key=nn.Linear(n_embd, head_size, bias=False)
        self.query=nn.Linear(n_embd, head_size, bias=False)
        self.value=nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C=x.shape
        k=self.key(x)   # (B,T,hs)
        q=self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei=q@k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei=wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei=F.softmax(wei, dim=-1)
        wei=self.dropout(wei)
        v=self.value(x)
        out=wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(head_size * num_heads,n_embd)
        self.dropout=nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size=n_embd // n_head
        self.sa=MultiHeadAttention(n_head, head_size)
        self.ffwd=FeedFoward(n_embd)
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table=nn.Embedding(block_size, n_embd)
        self.blocks=nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embd) # final layer norm
        self.lm_head=nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T =idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb=self.token_embedding_table(idx)
        pos_emb=self.position_embedding_table(torch.arange(T, device=device))
        x=tok_emb+pos_emb
        x=self.blocks(x)
        x=self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss=None
            perplexity=None
        else:
            B,T,C =logits.shape
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
            probabilities=F.softmax(logits, dim=-1)
            log_probabilities=torch.log(probabilities)
            log_prob=torch.gather(log_probabilities,1,targets.unsqueeze(1)).squeeze(1)
            N=targets.size(0)
            perplexity=torch.exp(-torch.sum(log_prob)/N)

        return logits, loss,perplexity

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond=idx[:, -block_size:]
            # get the predictions
            logits,loss,perplexity = self(idx_cond)
            # focus only on the last time step
            logits=logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs=F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next=torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx =torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
model = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
# checkpoint = torch.load('/content/drive/MyDrive/checkpoint_iteration_5000.pt') #map_location=device)
checkpoint = torch.load('/content/drive/MyDrive/checkpoint_iteration_5000.pt' , map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer =torch.optim.AdamW(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])



17.820737 M parameters


In [ ]:
context = torch.tensor([[7,58,10,6,3,26,10]],dtype=torch.long,device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))

-t:,$N:
Are is thought for that?

HORTENSIO:
Go on to take; I say a pair which lives
Then digestion by Camillo he can speak.

LUCENTIO:
A bawd, my lord! Then I bad be soft, and loves upon
me; and every above's constant and sovereign,
it takes a pair coaintry by John.

KING RICHARD II:
The sad indirect my behalf their heirs
And rank what we in your hateful eyes
Yield on or our one distress'd joy,
And yourselves, out and the fresh soul:
Think your heart's feed ears and what I have done
Your highn-goodly tongue, but the other predent,
Before I give to spend; and to your breaks
Your appredite are distinguined
And bitterly and your good horse with a gentleman
Wherein your good was he subder'd.

LEONTES:
Who ill--

FLORIZEL:
Then letters defend my master
He so from a learned preparating.

POLIXENES:
Beseech you, gentle master, fair lady,
Tell the senate, I was at gareful, end
'Tis general thought to give last their name;
But Romeo prosperousion; from the royal king's,
And mine ends a doubter